# Vulnerability Detection

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8" # this is needed to get rid of weird colab locale error
# if you are still running into issues, please restart the runtime to initialize a new environment

In [2]:
# obtain data
!wget https://github.com/uiuc-cs598lmz-s25/hw4/raw/main/d2a.zip

--2025-04-18 01:18:00--  https://github.com/uiuc-cs598lmz-s25/hw4/raw/main/d2a.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/uiuc-cs598lmz-s25/hw4/main/d2a.zip [following]
--2025-04-18 01:18:00--  https://raw.githubusercontent.com/uiuc-cs598lmz-s25/hw4/main/d2a.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71226 (70K) [application/zip]
Saving to: ‘d2a.zip’

d2a.zip             100%[===================>]  69.56K  --.-KB/s    in 0.02s   

2025-04-18 01:18:01 (4.26 MB/s) - ‘d2a.zip’ saved [71226/71226]



In [3]:
!unzip d2a.zip

Archive:  d2a.zip
   creating: d2a/D2A_12517/
  inflating: d2a/D2A_12517/meta_data.json  
  inflating: d2a/D2A_12517/src.c     
   creating: d2a/D2A_12532/
  inflating: d2a/D2A_12532/meta_data.json  
  inflating: d2a/D2A_12532/src.c     
   creating: d2a/D2A_1290891/
  inflating: d2a/D2A_1290891/meta_data.json  
  inflating: d2a/D2A_1290891/src.c   
   creating: d2a/D2A_1291082/
  inflating: d2a/D2A_1291082/meta_data.json  
  inflating: d2a/D2A_1291082/src.c   
   creating: d2a/D2A_1291129/
  inflating: d2a/D2A_1291129/meta_data.json  
  inflating: d2a/D2A_1291129/src.c   
   creating: d2a/D2A_1291528/
  inflating: d2a/D2A_1291528/meta_data.json  
  inflating: d2a/D2A_1291528/src.c   
   creating: d2a/D2A_1291610/
  inflating: d2a/D2A_1291610/meta_data.json  
  inflating: d2a/D2A_1291610/src.c   
   creating: d2a/D2A_1291672/
  inflating: d2a/D2A_1291672/meta_data.json  
  inflating: d2a/D2A_1291672/src.c   
   creating: d2a/D2A_1291695/
  inflating: d2a/D2A_1291695/meta_data.json  
  

In [4]:
import torch

from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, T5ForConditionalGeneration

def load_codet5_model():
    tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-base")
    model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base-codexglue-defect").cuda()
    return model, tokenizer

model, tokenizer = load_codet5_model()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/828 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [5]:
# example test using the small model

code_snippets = """
#include <stdio.h>
int main()
{
    return 0;
}
"""

input_tensor = tokenizer.encode(code_snippets, return_tensors='pt').to("cuda")
x = model.generate(input_tensor, max_length=50, temperature=1)


In [6]:
import glob
import json

def prepare_vuln_dataset():
    vuln_dataset = []
    for file in glob.glob("d2a/*/src.c"):
        with open(file, "r") as f:
          src = f.read()

        with open(file.replace("src.c","meta_data.json"), "r") as f:
          x = json.load(f)

        x['src'] = src
        vuln_dataset.append(x)

    return vuln_dataset


vuln_dataset = prepare_vuln_dataset()

In [7]:
import torch
from sklearn.metrics import precision_recall_fscore_support
from tqdm.auto import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def predict_vuln(code: str, model, tokenizer) -> bool:
    # 1) tokenize
    inputs = tokenizer(code, return_tensors="pt", truncation=True, padding=True).to(device)
    # 2) generate with greedy decoding
    with torch.no_grad():
        out_ids = model.generate(
            **inputs,
            max_length=4,    # enough to catch "0" or "1" or "vulnerable"
            num_beams=1,
            do_sample=False
        )
    raw = tokenizer.decode(out_ids[0], skip_special_tokens=True).strip()
    # 3) take the first “word” or character
    first = raw.split()[0].lower()
    # 4) map to bool
    if first in {"1", "true", "vulnerable", "yes"}:
        return True
    if first in {"0", "false", "safe", "no"}:
        return False
    # 5) fallback: print once so you can see what’s unexpected
    print(f"[WARN] unexpected model output → '{raw}'")
    return False


def codet5_vuln_detection(model, tokenizer, vuln_dataset) -> float:
    y_true, y_pred = [], []

    for example in tqdm(vuln_dataset, desc="CodeT5 vuln detection"):
        gt = bool(example["vulnerable"])
        pr = predict_vuln(example["src"], model, tokenizer)
        y_true.append(gt)
        y_pred.append(pr)

    _, _, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, pos_label=True, average="binary"
    )
    return f1

# Then run:
f1_score = codet5_vuln_detection(model, tokenizer, vuln_dataset)
print(f"CodeT5 F1 = {f1_score:.4f}")

CodeT5 vuln detection:   0%|          | 0/69 [00:00<?, ?it/s]

CodeT5 F1 = 0.1818


# Prompt-based Vulnerability Detection

In [8]:
import google.generativeai as genai
genai.configure(api_key="MY_KEY")
gemini = genai.GenerativeModel("models/gemini-2.0-flash")

In [9]:
import signal
import time
from google.generativeai import GenerationConfig

safety_settings = [ # google is afraid of pretty much everything i think.
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]


def create_gemini_config(
    max_tokens: int,
    temperature: float = 1,
    batch_size: int = 1,
):
    config = GenerationConfig(
        candidate_count=batch_size,
        max_output_tokens=max_tokens,
        temperature=temperature,
    )
    return config


def handler(signum, frame):
    # swallow signum and frame
    raise Exception("end of time")


def request_gemini_engine(model, message, config):
    ret = None
    count = 0
    while ret is None:
        try:
            signal.signal(signal.SIGALRM, handler)
            signal.alarm(100)
            ret = model.generate_content(message, generation_config=config, safety_settings=safety_settings)
            s = ret.text  # check if response can be accessed.
            signal.alarm(0)
        except Exception as e:
            # NOTE this exception handling is needed since sometimes gemini will
            # refuse to answer due to safety reason (even if all blockers are set off)
            # instead we just simply catch this and then retry the response.
            # don't be alarmed if certain inputs take a long time to finish
            # eventually it should return a response.
            ret = None  # reset
            signal.alarm(0)
            time.sleep(20)
    return ret

In [10]:
import signal
import time
from tqdm.auto import tqdm
from sklearn.metrics import precision_recall_fscore_support

def predict_vuln_gemini(code: str, gemini) -> bool:
    """
    Use Gemini (temperature=1) to predict whether `code` contains a vulnerability.
    Returns True if vulnerable, False otherwise.
    """
    # 1) configure Gemini
    config = create_gemini_config(
        max_tokens=100,
        temperature=1,
        batch_size=1,
    )

    # 2) build prompt with instructions
    prompt = (
        "Detect whether the following C code contains a security vulnerability. "
        "Answer *only* 'True' or 'False'.\n\n"
        "```c\n"
        f"{code}\n"
        "```"
    )

    # 3) call Gemini API
    ret = request_gemini_engine(gemini, prompt, config)
    resp = ret.text.strip().lower()

    # 4) parse the first token or word
    first = resp.split()[0]
    if first in {"true", "yes", "vulnerable", "1"}:
        return True
    if first in {"false", "no", "safe", "0"}:
        return False

    # fallback warning
    print(f"[WARN] unexpected Gemini output: '{resp}'")
    return False


def gemini_vuln_detection(gemini, vuln_dataset) -> float:
    """
    Iterate over vuln_dataset, use predict_vuln_gemini, and compute binary F1.
    """
    y_true = []
    y_pred = []

    for example in tqdm(vuln_dataset, desc="Gemini vuln detection"):
        gt = bool(example["vulnerable"])
        pr = predict_vuln_gemini(example["src"], gemini)

        y_true.append(gt)
        y_pred.append(pr)

    # compute F1 (positive label = vulnerable)
    _, _, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, pos_label=True, average="binary"
    )
    return f1


# Example usage:
gemini_f1_score = gemini_vuln_detection(gemini, vuln_dataset)
print(f"Gemini F1 = {gemini_f1_score:.4f}")

Gemini vuln detection:   0%|          | 0/69 [00:00<?, ?it/s]

Gemini F1 = 0.5067


In [12]:
from tqdm.auto import tqdm

# Define the allowed labels
MULTI_LABELS = [
    "Integer-Overflow",
    "Buffer-Overflow",
    "Null-Pointer-Dereference",
    "Resource-Exhaustion-Error",
    "No-Vulnerability",
]

def predict_vuln_gemini_multi(code: str, gemini) -> str:
    """
    Use Gemini (temperature=1) to classify `code` into one of the MULTI_LABELS.
    Returns exactly one of those label strings.
    """
    config = create_gemini_config(
        max_tokens=50,
        temperature=1,
        batch_size=1,
    )

    prompt = (
        "Classify the following C code snippet into exactly one of these categories:\n"
        + ", ".join(MULTI_LABELS)
        + "\n\n```c\n"
        + code
        + "\n```\n\n"
        "Answer with *only* the category name."
    )

    ret = request_gemini_engine(gemini, prompt, config)
    resp = ret.text.strip()

    # Normalize and find which label appears
    for label in MULTI_LABELS:
        if label.lower() in resp.lower():
            return label

    # Fallback: maybe the model returned an index
    first = resp.split()[0]
    if first.isdigit():
        idx = int(first)
        if 0 <= idx < len(MULTI_LABELS):
            return MULTI_LABELS[idx]

    print(f"[WARN] unexpected multi-class output: '{resp}'")
    return "No-Vulnerability"  # safe fallback


def gemini_vuln_detection_multi(gemini, vuln_dataset) -> list:
    """
    Runs multi-class classification on vuln_dataset,
    returns a list of per-class accuracies in the same order as MULTI_LABELS.
    """
    # initialize counts
    counts = {lbl: 0 for lbl in MULTI_LABELS}
    correct = {lbl: 0 for lbl in MULTI_LABELS}

    for example in tqdm(vuln_dataset, desc="Gemini multi-class detection"):
        # unwrap the ground-truth label
        gt_raw = example["vulnerability_type"]
        if isinstance(gt_raw, list):
            gt = gt_raw[0] if gt_raw else "No-Vulnerability"
        else:
            gt = gt_raw

        # run prediction
        pred = predict_vuln_gemini_multi(example["src"], gemini)

        # tally
        if gt in counts:
            counts[gt] += 1
            if pred == gt:
                correct[gt] += 1
        else:
            # unexpected label in dataset
            print(f"[WARN] unknown GT label: {gt}")

    # compute per-class accuracy
    accuracies = []
    for lbl in MULTI_LABELS:
        if counts[lbl] > 0:
            accuracies.append(correct[lbl] / counts[lbl])
        else:
            accuracies.append(0.0)

    return accuracies


# Usage:
gemini_multi_class_vulnerability = gemini_vuln_detection_multi(gemini, vuln_dataset)
print("Per-class accuracy:", gemini_multi_class_vulnerability)

Gemini multi-class detection:   0%|          | 0/69 [00:00<?, ?it/s]

Per-class accuracy: [0.23076923076923078, 0.5, 0.0, 0.0, 0.6122448979591837]
